In [1]:

import os
import polars as pl
from polars import col as c
import numpy as np


# from distflow_algorithm import DistFlow
import time
import pandapower as pp


from data_connector import pandapower_to_distflow
from general_function import duckdb_to_dict, dict_to_duckdb, pl_to_dict
from networkx_function import (
    get_all_edge_data,
    generate_shortest_path_length_matrix,
    generate_bfs_tree_with_edge_data,
    generate_nx_edge,
    generate_tree_graph_from_edge_data,
)


from distflow_algorithm import DistFlow


# Useless outside jupiternotebook because in settings.py a line that changes the directory to src for ipynb
os.chdir(os.getcwd().replace("/src", ""))

In [2]:
ext_grid_id = "1"
n_1 = 0.95
n_2 = 1.02
v_ext_grid_sq = 1.05

line_data: pl.DataFrame = pl.DataFrame(
    {
        "line_id": np.arange(1, 13).astype(str),
        "type": ["branch"] * 10 + ["transformer"] * 2,
        "u_of_edge": ["2", "2", "2", "4", "5", "6", "7", "10", "12", "13", "8", "9"],
        "v_of_edge": ["11", "1", "3", "1", "4", "4", "8", "9", "11", "11", "4", "8"],
        "n_transfo": [1.0] * 10 + [n_1**2, n_2**2],
        "x_pu": np.arange(1, 13) * 2e-3,
        "r_pu": np.arange(1, 13) * 1e-3,
        "b_pu": list(np.arange(1, 11) * -1e-3) + [0.001, 0.001],
        "g_pu": [0.0] * 10 + [0.001, 0.001],
    }
)

node_data: pl.DataFrame = pl.DataFrame(
    {
        "node_id": np.arange(1, 14).astype(str),
        "v_base": [400.0] * 7 + [200.0, 400.0, 400.0, 200.0, 100.0, 100.0],
        "p_node_pu": np.array([0, 10, 2, 0, 0, -2, 0, 7, 25, 0, 100, 0.5, 10]) * 1e-2,
        "q_node_pu": np.array([0, 0, 0.2, 2, 0, -0.2, 0, 0.7, 0, 0, 0, 0.5, 1]) * 1e-2,
    },
    strict=False,
)

np.set_printoptions(linewidth=200)
distflow = DistFlow(line_data=line_data, ext_grid_id=ext_grid_id)
node_data = node_data.with_columns(
    c("node_id")
    .replace_strict(distflow.node_id_to_nb_mapping, default=None)
    .alias("idx")
).sort("idx")
v0_sqr = distflow.v_in_sqr_np * v_ext_grid_sq

s_node = node_data["p_node_pu"].to_numpy() + 1j * node_data["q_node_pu"].to_numpy()

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(s_node=s_node, v0_sqr=v0_sqr, engine="numpy")
print("numpy:", time.time() - tic)

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(
    s_node=s_node, v0_sqr=v0_sqr, engine="graphblas"
)
print("graphblas:", time.time() - tic)

numpy: 0.00034880638122558594
graphblas: 0.0075342655181884766


n_transfo

In [11]:
s_base = 1e6
net: pp.pandapowerNet = pp.from_pickle("data/input_grid/modified_cigre_network_lv.p")
tic = time.time()
pp.runpp(net)
print("f:", time.time() - tic)


f: 0.012510061264038086


In [4]:

ext_grid_id: str = str(net.ext_grid["bus"][0])
v_ext_grid_sq: float = net.ext_grid["vm_pu"][0] ** 2
node_data, line_data = pandapower_to_distflow(net=net, s_base=s_base)
distflow: DistFlow = DistFlow(line_data=line_data, ext_grid_id=ext_grid_id)

node_data = node_data.with_columns(
    c("node_id")
    .replace_strict(distflow.node_id_to_nb_mapping, default=None)
    .alias("idx")
).sort("idx")
v0_sqr= distflow.v_in_sqr_np * v_ext_grid_sq
s_node = node_data["p_node_pu"].to_numpy() + 1j * node_data["q_node_pu"].to_numpy()

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(s_node=s_node, v0_sqr=v0_sqr, engine="numpy")
print("numpy:", time.time() - tic)

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(
    s_node=s_node, v0_sqr=v0_sqr, engine="graphblas"
)
print("graphblas:", time.time() - tic)

numpy: 0.0006625652313232422
graphblas: 0.026476144790649414


In [5]:
line_res_pp = pl.DataFrame(
    {
        "node_id": list(net.line["to_bus"].astype(str)),
        "p_pp": list(net.res_line["p_from_mw"]),
        "q_pp": list(net.res_line["q_from_mvar"]),
    }
)

node_res_pp = pl.from_pandas(net.res_bus["vm_pu"], include_index=True).select(
    c("index").cast(pl.Utf8).alias("node_id"), "vm_pu"
)

result = (
    pl.DataFrame(
        {
            "v": np.sqrt(v),
            "p": np.real(s_flow),
            "q": np.imag(s_flow),
        }
    )
    .with_row_index(name="node_id")
    .with_columns(
        c("node_id")
        .replace_strict(distflow.node_nb_to_id_mapping, default=None)
        .alias("node_id")
    )
    .join(line_res_pp, on="node_id", how="left")
    .join(node_res_pp, on="node_id", how="left")
    .with_columns(
        ((c("v") - c("vm_pu")) / c("vm_pu")).abs().alias("diff_v"),
        ((c("p") - c("p_pp")) / c("p_pp")).abs().alias("diff_p"),
        ((c("q") - c("q_pp")) / c("p_pp")).abs().alias("diff_q"),
    )
)
print(result["diff_v", "diff_p", "diff_q"].max())

shape: (1, 3)
┌──────────┬──────────┬──────────┐
│ diff_v   ┆ diff_p   ┆ diff_q   │
│ ---      ┆ ---      ┆ ---      │
│ f64      ┆ f64      ┆ f64      │
╞══════════╪══════════╪══════════╡
│ 0.000392 ┆ 0.000182 ┆ 0.010306 │
└──────────┴──────────┴──────────┘


In [6]:
net: pp.pandapowerNet = pp.from_pickle("data/input_grid/synthesized_grid.p")
net.trafo["i0_percent"] = 2


In [7]:
tic = time.time()
pp.runpp(net)
print("f:", time.time() - tic)

f: 0.5805337429046631


In [8]:
s_base = 1e6
ext_grid_id: str = str(net.ext_grid["bus"][0])
v_ext_grid_sq: float = net.ext_grid["vm_pu"][0] ** 2

node_data, line_data = pandapower_to_distflow(net=net, s_base=s_base)
distflow = DistFlow(line_data=line_data, ext_grid_id=ext_grid_id)

node_data = node_data.with_columns(
    c("node_id")
    .replace_strict(distflow.node_id_to_nb_mapping, default=None)
    .alias("idx")
).sort("idx")
v0_sqr = distflow.v_in_sqr_np * v_ext_grid_sq
s_node = node_data["p_node_pu"].to_numpy() + 1j * node_data["q_node_pu"].to_numpy()



In [9]:

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(
    s_node=s_node, v0_sqr=v0_sqr, engine="graphblas"
)
print("graphblas:", time.time() - tic)


graphblas: 0.1157536506652832
